In [ ]:
import shutil
import rasterio
import numpy as np
import rasterio.warp
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import tqdm

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import mysecrets
import fetch_from_cluster
import create_datacube
import rsutils.utils

In [ ]:
remotepath = '/gpfs/data1/cmongp2/sasirajann/fetch_satdata/data/etiopia/s2gridid/164c59c.zip'

zip_filename_wo_ext = remotepath.split('/')[-1].removesuffix('.zip')
stack_folderpath = f'../data/stack_{zip_filename_wo_ext}'
tif_folderpath = f'../data/tifs_{zip_filename_wo_ext}'

In [ ]:
zip_filepath = fetch_from_cluster.download_file_from_cluster(
    sshcreds = mysecrets.SSH_UMD_SASIRAJANN,
    remotepath = remotepath,
    download_folderpath = '../data/cluster_files',
    # overwrite = True,
)

shutil.unpack_archive(zip_filepath, stack_folderpath)

datacube, metadata = create_datacube.load_datacube(stack_folderpath)

In [ ]:
datacube.shape

In [ ]:
def write_all_tifs_out(
    datacube:np.ndarray, 
    bandnames:list[str], 
    out_meta:dict, 
    folderpath:str,
):
    n_ts, _, _, _ = datacube.shape
    band_indices = {bandname:index for index, bandname in enumerate(bandnames)}

    filepaths = {}

    os.makedirs(folderpath, exist_ok=True)

    for ts in range(n_ts):
        filepaths[ts] = {}
        
        for bandname, band_index in band_indices.items():
            tif_filepath = os.path.join(folderpath, f'{bandname}_{str(ts).zfill(3)}.tif')
            
            with rasterio.open(tif_filepath, 'w', **out_meta) as dst:
                dst.write(np.expand_dims(datacube[ts,:,:,band_index], axis=0))
            
            filepaths[ts][bandname] = tif_filepath
    
    return filepaths

In [ ]:
metadata

In [ ]:
write_all_tifs_out(
    datacube = datacube,
    out_meta = metadata['geotiff_metadata'],
    folderpath = tif_folderpath,
    bandnames = metadata['bands'],
)